In [ ]:
from langchain_google_vertexai import ChatVertexAI
from langchain_core.prompts import PromptTemplate

llm = ChatVertexAI(model="gemini-2.0-flash-001")

# ReACT example

Let's deep dive into the ReACT pattern and implement it ourselves. We will mock tools as simple Python functions (without actual implementation) for now:

In [ ]:
import math

def mocked_google_search(query: str) -> str:
  print(f"CALLED GOOGLE_SEARCH with query={query}")
  return "Donald Trump is a president of USA and he's 78 years old"

def mocked_calculator(expression: str) -> float:
  print(f"CALLED CALCULATOR with expression={expression}")
  if "sqrt" in expression:
    return math.sqrt(78*132)
  return 78*132

Let's define schemas for our tools and pass them to an LLM:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

calculator_tool = {
   "title": "calculator",
    "description": "Computes mathematical expressions",
   "type": "object",
   "properties": {
       "expression": {
           "description": "A mathematical expression to be evaluated by a calculator",
           "title": "expression",
           "type": "string"},
   },
   "required": ["expression"]
}

search_tool = {
    "description": "Returns about common facts, fresh events and news from Google Search engine based on a query.",
    "title": "google_search",
    "type": "object",
    "properties": {
        "query": {
            "description": "Search query to be sent to the search engine",
            "title": "search_query",
            "type": "string"},
    },
    "required": ["query"]
}

system_prompt = (
    "Always use a calculator for mathematical computations, and use Google Search "
    "for information about common facts, fresh events and news. Do not assume anything, keep in "
    "mind that things are changing and always "
    "check yourself with external sources if possible."
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="messages"),
])


llm_with_tools = llm.bind(tools=[search_tool, calculator_tool]).bind(prompt=prompt)

Now it's time to define the LangGraph workflow itself.

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage
from langgraph.graph import MessagesState, StateGraph, START, END


def invoke_llm(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


def call_tools(state: MessagesState):
    last_message = state["messages"][-1]
    tool_calls = last_message.tool_calls

    new_messages = []

    for tool_call in tool_calls:
      if tool_call["name"] == "google_search":
        tool_result = mocked_google_search(**tool_call["args"])
        new_messages.append(ToolMessage(content=tool_result, tool_call_id=tool_call["id"]))
      elif tool_call["name"] == "calculator":
        tool_result = mocked_calculator(**tool_call["args"])
        new_messages.append(ToolMessage(content=tool_result, tool_call_id=tool_call["id"]))
      else:
        raise ValueError(f"Tool {tool_call['name']} is not defined!")
    return {"messages": new_messages}


def should_run_tools(state: MessagesState):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
      return "call_tools"
    return END

And we can put everything together as following:

In [ ]:
builder = StateGraph(MessagesState)
builder.add_node("invoke_llm", invoke_llm)
builder.add_node("call_tools", call_tools)

builder.add_edge(START, "invoke_llm")
builder.add_conditional_edges("invoke_llm", should_run_tools)
builder.add_edge("call_tools", "invoke_llm")
graph = builder.compile()

In [ ]:
question = "What is a square root of the current US president’s age multiplied by 132?"

result = graph.invoke({"messages": [HumanMessage(content=question)]})
print(result["messages"][-1].content)

CALLED GOOGLE_SEARCH with query=age of Joe Biden
CALLED CALCULATOR with expression=sqrt(78 * 132)
The square root of the current US president’s age multiplied by 132 is approximately 101.47.



Now as you understand how it works under the hood, we can share good news with you. There's no need to implement it yourselves, you can use an customizable pre-built agent from LangGraph:

In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(llm=llm, tools=[search_tool, calculator_tool], prompt=system_prompt)

AIMessage(content="The square root of the current US president's age (78) multiplied by 132 is approximately 102.", additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 117, 'candidates_token_count': 28, 'total_token_count': 145, 'cached_content_token_count': 0}, 'finish_reason': 'STOP', 'avg_logprobs': -0.046135395765304565}, id='run-a29bf725-d310-4643-9482-6ec223853c80-0', usage_metadata={'input_tokens': 117, 'output_tokens': 28, 'total_tokens': 145})